In [1]:
# Some of the methods for this problem were reused from homework 5

import pickle
from keras.models import Sequential
from keras.layers import Dense, LSTM, Activation
import numpy as np
from keras import optimizers

Using TensorFlow backend.


In [2]:
letter_data = pickle.load(open("data/letter_data.p", "rb"))

In [3]:
# Dictionaries to convert between letter and index
letter_int = {}
int_letter = {}
i = 0
for poem in letter_data:
    for letter in poem:
        if letter not in letter_int:
            letter_int[letter] = i
            int_letter[i] = letter
            i += 1

In [4]:
# Returns one-hot-encoded feature representation of the specified word given
# a dictionary mapping words to their one-hot-encoded index.
def get_word_repr(letter_to_int, word):
    unique_words = letter_to_int.keys()
    # Return a vector that's zero everywhere besides the index corresponding to <word>
    feature_representation = np.zeros(len(unique_words))
    feature_representation[letter_to_int[word]] = 1
    return feature_representation

In [5]:
def generate_traindata(word_list, word_to_index, window_size=40, skip = 5):
    """
    Generates training data for Skipgram model (sort of).

    Arguments:
        word_list:     Sequential list of letters (strings).
        word_to_index: Dictionary mapping words to their corresponding index
                       in a one-hot-encoded representation of our corpus.

        window_size:   Size of Skipgram window.
        
        skip:          Skip every skip characters 

    Returns:
        (trainX, trainY):     A pair of matrices (trainX, trainY) containing training
                              points (one-hot-encoded vectors representing individual words) and
                              their corresponding labels (also one-hot-encoded vectors representing words).
    """
    trainX = []
    trainY = []
    for i in range(window_size, len(word_list), skip):
        curr_word = word_list[i]
        curr_X = []
        for j in range(-window_size, 0):
            if j != 0 and i + j >= 0 and i + j < len(word_list):
                adjacent_word = word_list[i + j]
                curr_X.append(get_word_repr(word_to_index, adjacent_word))
        trainX.append(curr_X)
        trainY.append(get_word_repr(word_to_index, curr_word))
        
    return (np.array(trainX), np.array(trainY))

In [6]:
# Create the training set
unit = True
train_x = -1
train_y = -1
for poem in letter_data:
    poem_train_x, poem_train_y = generate_traindata(poem, letter_int)
    if unit:
        train_x = poem_train_x
        train_y = poem_train_y
        unit = False
    else:
        train_x = np.concatenate((train_x, poem_train_x))
        train_y = np.concatenate((train_y, poem_train_y))

In [7]:
# Generate the neural network
model = Sequential()
model.add(LSTM(100, input_shape = (len(train_x[0]), len(train_x[0][0]))))
model.add(Dense(len(train_y[0])))
model.add(Activation('softmax'))
model.summary()

model.compile(loss = "categorical_crossentropy", optimizer = "rmsprop", metrics = ['accuracy'])
model.fit(train_x, train_y, epochs = 200, batch_size = 128)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 100)               55600     
_________________________________________________________________
dense_1 (Dense)              (None, 38)                3838      
_________________________________________________________________
activation_1 (Activation)    (None, 38)                0         
Total params: 59,438
Trainable params: 59,438
Non-trainable params: 0
_________________________________________________________________
Epoch 1/200
17573/17573 [==============================] - 8s 481us/step - loss: 3.0030 - acc: 0.1627
Epoch 2/200
17573/17573 [==============================] - 8s 454us/step - loss: 2.8252 - acc: 0.2086
Epoch 3/200
17573/17573 [==============================] - 8s 467us/step - loss: 2.5915 - acc: 0.2822
Epoch 4/200
17573/17573 [==============================] - 8s 456us/step - loss: 2.4263 - acc: 0

17573/17573 [==============================] - 8s 441us/step - loss: 0.5984 - acc: 0.8351
Epoch 74/200
17573/17573 [==============================] - 8s 440us/step - loss: 0.5847 - acc: 0.8374
Epoch 75/200
17573/17573 [==============================] - 8s 442us/step - loss: 0.5667 - acc: 0.8472
Epoch 76/200
17573/17573 [==============================] - 8s 442us/step - loss: 0.5525 - acc: 0.8524
Epoch 77/200
17573/17573 [==============================] - 8s 440us/step - loss: 0.5428 - acc: 0.8515
Epoch 78/200
17573/17573 [==============================] - 8s 445us/step - loss: 0.5263 - acc: 0.8585
Epoch 79/200
17573/17573 [==============================] - 8s 469us/step - loss: 0.5154 - acc: 0.8599
Epoch 80/200
17573/17573 [==============================] - 9s 502us/step - loss: 0.5057 - acc: 0.8617
Epoch 81/200
17573/17573 [==============================] - 8s 483us/step - loss: 0.4884 - acc: 0.8700
Epoch 82/200
17573/17573 [==============================] - 8s 479us/step - loss: 0.47

17573/17573 [==============================] - 8s 437us/step - loss: 0.1645 - acc: 0.9622
Epoch 153/200
17573/17573 [==============================] - 8s 436us/step - loss: 0.1572 - acc: 0.9631
Epoch 154/200
17573/17573 [==============================] - 8s 435us/step - loss: 0.1627 - acc: 0.9620
Epoch 155/200
17573/17573 [==============================] - 8s 435us/step - loss: 0.1524 - acc: 0.9657
Epoch 156/200
17573/17573 [==============================] - 8s 436us/step - loss: 0.1567 - acc: 0.9647
Epoch 157/200
17573/17573 [==============================] - 8s 436us/step - loss: 0.1504 - acc: 0.9653
Epoch 158/200
17573/17573 [==============================] - 8s 436us/step - loss: 0.1508 - acc: 0.9670
Epoch 159/200
17573/17573 [==============================] - 8s 437us/step - loss: 0.1497 - acc: 0.9660
Epoch 160/200
17573/17573 [==============================] - 8s 438us/step - loss: 0.1445 - acc: 0.9682
Epoch 161/200
17573/17573 [==============================] - 8s 435us/step - l

In [15]:
model.fit(train_x, train_y, epochs = 200, batch_size = 128)

Epoch 1/200
17573/17573 [==============================] - 8s 449us/step - loss: 0.1128 - acc: 0.9738
Epoch 2/200
17573/17573 [==============================] - 8s 454us/step - loss: 0.1152 - acc: 0.9724
Epoch 3/200
17573/17573 [==============================] - 8s 451us/step - loss: 0.1171 - acc: 0.9717
Epoch 4/200
17573/17573 [==============================] - 8s 450us/step - loss: 0.1161 - acc: 0.9720
Epoch 5/200
17573/17573 [==============================] - 8s 451us/step - loss: 0.1105 - acc: 0.9758
Epoch 6/200
17573/17573 [==============================] - 8s 457us/step - loss: 0.1123 - acc: 0.9736
Epoch 7/200
17573/17573 [==============================] - 8s 454us/step - loss: 0.1118 - acc: 0.9752
Epoch 8/200
17573/17573 [==============================] - 8s 456us/step - loss: 0.1113 - acc: 0.9745
Epoch 9/200
17573/17573 [==============================] - 8s 461us/step - loss: 0.1134 - acc: 0.9738
Epoch 10/200
17573/17573 [==============================] - 8s 457us/step - loss: 

17573/17573 [==============================] - 8s 437us/step - loss: 0.0901 - acc: 0.9779
Epoch 81/200
17573/17573 [==============================] - 8s 437us/step - loss: 0.0984 - acc: 0.9747
Epoch 82/200
17573/17573 [==============================] - 8s 435us/step - loss: 0.0874 - acc: 0.9807
Epoch 83/200
17573/17573 [==============================] - 8s 436us/step - loss: 0.0848 - acc: 0.9788
Epoch 84/200
17573/17573 [==============================] - 8s 436us/step - loss: 0.0922 - acc: 0.9770
Epoch 85/200
17573/17573 [==============================] - 8s 436us/step - loss: 0.0876 - acc: 0.9772
Epoch 86/200
17573/17573 [==============================] - 8s 443us/step - loss: 0.0853 - acc: 0.9789
Epoch 87/200
17573/17573 [==============================] - 8s 451us/step - loss: 0.0856 - acc: 0.9798
Epoch 88/200
17573/17573 [==============================] - 8s 454us/step - loss: 0.0910 - acc: 0.9776
Epoch 89/200
17573/17573 [==============================] - 8s 450us/step - loss: 0.08

17573/17573 [==============================] - 8s 447us/step - loss: 0.0748 - acc: 0.9804
Epoch 160/200
17573/17573 [==============================] - 8s 451us/step - loss: 0.0775 - acc: 0.9794
Epoch 161/200
17573/17573 [==============================] - 8s 450us/step - loss: 0.0765 - acc: 0.9807
Epoch 162/200
17573/17573 [==============================] - 8s 450us/step - loss: 0.0784 - acc: 0.9794
Epoch 163/200
17573/17573 [==============================] - 8s 448us/step - loss: 0.0823 - acc: 0.9781
Epoch 164/200
17573/17573 [==============================] - 8s 448us/step - loss: 0.0724 - acc: 0.9818
Epoch 165/200
17573/17573 [==============================] - 8s 449us/step - loss: 0.0733 - acc: 0.9820
Epoch 166/200
17573/17573 [==============================] - 8s 447us/step - loss: 0.0760 - acc: 0.9802
Epoch 167/200
17573/17573 [==============================] - 8s 447us/step - loss: 0.0767 - acc: 0.9802
Epoch 168/200
17573/17573 [==============================] - 8s 446us/step - l

In [16]:
# Convert a line to one hot form
def convert_line_to_one_hot(line, letter_int):
    output = []
    for letter in line:
        output.append(get_word_repr(letter_int, letter))
    return np.array(output)

In [17]:
# Convert from one hot to letter
def vect_to_letter(vect, int_letter):
    max_index = 0
    max_value = 0
    for curr_index, curr_value in enumerate(vect):
        if max_value < curr_value:
            max_index = curr_index
            max_value = curr_value
    return int_letter[max_index]

In [18]:
# Given a RNN generates a shakespeare sonnet
def generate_poem(RNN, letter_int, int_letter):
    # Initial line to seed 
    first_line = "shall i compare thee to a summer\'s day?\n"
    
    curr_line = convert_line_to_one_hot(first_line, letter_int)
    curr_poem = first_line
    line_num = 2
    while line_num < 14:
        # Predict the next character
        next_char = RNN.predict(np.array([curr_line]))[0]
        # Update curr_line and curr_poem
        curr_poem += vect_to_letter(next_char, int_letter)
        if vect_to_letter(next_char, int_letter) == "\n":
            line_num += 1
        curr_line = convert_line_to_one_hot(curr_poem[-40:], letter_int)
    
    return curr_poem

In [19]:
poem = generate_poem(model, letter_int, int_letter)
print(poem)

shall i compare thee to a summer's day?
the io al ond rovel this hight praceed no cooke.
sow ceme thy cort, wor with fratt far love,
and yout my send and all the arpound,
and that thenedest tor, wellds pormous toudd.
be'ing nit eve shuch cone that brosed no,
and and that is on ollle's sprice,
and so grack and theis farture as with me,
the fir thene shace as end thy hrowed,
that i hournele sucround of decigeincy ing.
art;
e cornul abk im monese whan thy vesert,
pood to thee wre whech the forr would sulf can,



'h'